# Save Blackrock iEEG files to EDF format following BIDS structure

Este script permite tomar los datos originales en el formato de salida del sistema Blackrock (.nev, .ns3, ..) y guardarlo en otro directorio que sigue la estructura BIDS. Se utilizará el formato EDF para guardar los archivos de iEEG.

Además, con ayuda de BIDS, se guardarán archivos accesorios con los eventos que fueron enviados durante la tarea, para posteriormente levantarlos con MNE-Python (u otro software)

**IMPORTANTE**

En los registros que tomo pueden pasar dos cosas: los correspondiente a todas las tareas evaluadas (VMA, MSL) se encuentra en un único archivo o cada tarea tiene su archivo por separado. Por lo tanto, este script me permite también separar las tareas en caso de ser necesario. Eso se realiza a mano indicando el punto en el cual producir la division.

Otro inconveniente que puede suceder es que haya eventos incorrectos o repetidos (por negligencia mia). Hay algunas lineas de codigos dedicadas a solucionar esos inconvenientes. --> ESO ESTA EN OTRO SCRIPT

**CHEQUEAR TODOS LOS PASOS!!**

### Librerias

In [1]:
%matplotlib qt5

In [2]:
from neo.rawio import BlackrockRawIO
import mne
import numpy as np
import matplotlib.pyplot as plt
import mne_bids 
import mne_bids.utils 

## Funciones custom

In [3]:
# Load .nev3 Blackrock file, return neo reader
def load_original_ieeg_nev3(filename):
    """ Funcion para levantar una senal iEEG de Blackrock tipo nev3.
    filename : path y nombre completo del archivo a levantar
    Return: reader de la libreria 'neo' 
     """
    reader = BlackrockRawIO(filename=filename, nsx_to_load=3)
    reader.parse_header()
    return reader

# Look for a specific sequence in numpy array
def search_sequence_numpy(arr,seq):
    """ Find sequence in an array using NumPy only.

    Parameters
    ----------    
    arr    : input 1D array
    seq    : input 1D array

    Output
    ------    
    Output : 1D Array of indices in the input array that satisfy the 
    matching of input sequence in the input array.
    In case of no match, an empty list is returned.

    Source: https://stackoverflow.com/questions/36522220/searching-a-sequence-in-a-numpy-array
    """

    # Store sizes of input array and sequence
    Na, Nseq = arr.size, seq.size

    # Range of sequence
    r_seq = np.arange(Nseq)

    # Create a 2D array of sliding indices across the entire length of input array.
    # Match up with the input sequence & get the matching starting indices.
    M = (arr[np.arange(Na-Nseq+1)[:,None] + r_seq] == seq).all(1)

    # Get the range of those indices as final output
    if M.any() >0:
        return np.where(np.convolve(M,np.ones((Nseq),dtype=int))>0)[0]
    else:
        return []

# Create mne raw dataset from neo reader
def create_mne_dataset_from_neoReader(reader,ev_timestamps_sfreq=30000):
    """ Create mne dataset from neo reader 
    reader: objeto reader de la libreria neo
    ev_timestamps_sfreq: "frecuencia de muestreo" o resolucion temporal del canal de los eventos digitales
    Asume: block=0, segment=0, time_start=0, time_stop=fin del archivo
    Return: raw signal en Volts
    """

    #Extraigo las senales de todos los canales, de todo el tiempo
    raw_signal = reader.get_analogsignal_chunk(block_index=0, seg_index=0, i_start=0, i_stop=None)
    raw_signal = raw_signal*(1e-6) # --> para convertir a Volts
    sampling_rate = reader.get_signal_sampling_rate()
    
    #Extraigo los nombres de los canales del reader
    ch_names = [reader.header["signal_channels"][idx][0] for idx in range(raw_signal.shape[1])]
    ch_types = ['seeg'] * len(ch_names)  

    #Se indican los canales de tipo ECG, en caso de haber sido registrados
    indx_ECG_chann = [i for i, s in enumerate(ch_names) if 'ECG' in s]
    for i in indx_ECG_chann:
        ch_types[i] = 'ecg' 
    
    #Info mne
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sampling_rate)
    
    #Creacion del dataset mne
    raw_mne = mne.io.RawArray(raw_signal.T, info)

    #Creacion de canal para incluir la informacion de eventos
    #Originalmente es una senal digital que se transforma a analogica para crear el dataset de mne
    stim_chann = np.zeros((1, raw_signal.T.shape[1]))
    info_stim = mne.create_info(['STIM'], raw_mne.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_chann, info_stim)
    raw_mne.add_channels([stim_raw], force_update_info=True)

    #Inclusion de los eventos a partir de la informacion del registro original
    ev_timestamps, ev_durations, ev_labels = reader.get_event_timestamps(block_index=0, seg_index=0, event_channel_index=0,
                    t_start=None, t_stop=None)

    ev_sample = np.floor(ev_timestamps*sampling_rate/ev_timestamps_sfreq).astype(int)
    #Se conservan los 8 bits con el codigo del evento
    ev_labels_num = ev_labels.astype(int) & 0xff
    
    # Ambas tareas
    # 2 --> 64: Inicio Baseline --> en el registro original es 2, lo transformo a 64
    # 8 --> 128: Fin Baseline  --> en el registro original es 8, lo transformo a 128

    # Eventos para la tarea MSL:
    # 4: Inicio REST
    # 16: Inicio TASK

    # Eventos para la tarea VMA:
    # 4: Inicio TRIAL
    # 8: Inicio MOVIMIENTO
    # 16: Fin MOVIMIENTO
    # 32: Fin TRIAL

    posibles_events = [2, 8, 4, 16, 32]
    valid_events = np.isin(ev_labels_num,posibles_events)
    events_array = np.block([[ev_sample[valid_events]], [np.zeros(np.sum(valid_events))], [ev_labels_num[valid_events]]]).T
    events_array = events_array.astype(int)

    #Transformo los codigos de los eventos correspondientes a Inicio y Fin de Baseline porque coinciden con otros eventos
    #   1. Se busca los indices de la ocurrencia de la secuencia 2-8, que indican en el inicio y fin del periodo de basline/rest
    #   2. Reemplzaso la secuencia 2-8 por la secuencia 64-128
    old_seq = [2,8]
    new_seq = [64,128]
    indxs_seq = search_sequence_numpy(events_array[:,2],np.array(old_seq))
    events_array[indxs_seq,2] = np.tile(np.array(new_seq), int(indxs_seq.size/2))

    
    # ESTO ES SOLO PARA EL SUJETO 05, QUE LE FALTA UN TRIGGER DE FIN DE BASELINE en el tarea VMA
    #events_array[-1][2] = 64
    #events_array = np.row_stack((events_array,np.array([raw_mne.n_times-1, 0, 128])))

    # Se agrega la informacion de los eventos como canal STIM
    raw_mne.add_events(events_array, stim_channel='STIM', replace=True)

    return raw_mne

# Export/save .edf file from mne raw dataset
def export_mne_to_edf_BIDSformat(raw_mne_in, root, subject, task_name, session_name, run, datatype, description = None, extension='.edf'):
    """ Export mne dataset to .edf file using BIDS format
    raw_mne: mne data set
    root: BIDS root path
    subject: subject name/code
    task: task name
    session: session name
    run: run number
    datatype: BIDS datatype
    extension: por defecto la extension es .edf
    """

    #subject = '07' 
    #task_name = task['vma']
    #session_name = session['ses1']
    #run = None
    #datatype = 'ieeg'

    events = mne.find_events(raw_mne_in, shortest_event=1, stim_channel='STIM')

    if task_name == 'vma':

        #Se filtran los eventos para consevar solo los que corresponden a la tarea, en caso de que haya alguno incorrecto
        posibles_events_vma = [4, 8, 16, 32, 64, 128]     
        valid_events_vma = np.isin(events[:,2],posibles_events_vma)
        events = events[valid_events_vma,:]
        events_id = {'start_bsl_rest': 64, 'stop_bsl_rest': 128, 'start_trial': 4, 'start_mov': 8, 'end_mov': 16, 'end_trial': 32}
    
    elif task_name == 'msl':
        
        #Se filtran los eventos para consevar solo los que corresponden a la tarea, en caso de que haya alguno incorrecto
        posibles_events_msl = [4, 16, 64, 128]     
        valid_events_msl = np.isin(events[:,2],posibles_events_msl)
        events = events[valid_events_msl,:]
        events_id = {'start_bsl_rest': 64, 'stop_bsl_rest': 128, 'inicio_msl_rest': 4, 'inicio_msl_task': 16}

    BIDS_path = mne_bids.BIDSPath(root=root, subject=subject, task=task_name, session=session_name,
                run=run, datatype=datatype, extension=extension, description = description)

    raw_mne_in.drop_channels('STIM')

    #Gardar el archivo tipo .edf siguiendo el formato BIDS
    mne_bids.write_raw_bids(raw_mne_in, BIDS_path, format='EDF', symlink=False, empty_room=None, 
        allow_preload=True, overwrite=True, verbose=True, events = events, event_id = events_id)

    #####
    #bids_path = mne_bids.BIDSPath(root=root, subject=subject, task=task, session=session,
    #                     run=run, datatype=datatype, extension=extension)
                           
    #mne.export.export_raw(bids_path.fpath, raw_mne, fmt='auto', physical_range='auto', add_ch_type=False, overwrite=False, verbose=None)
    
    return BIDS_path

#Split mne dataset according to specific (start/stop) indexes 
def split_mne_dataset(raw_mne_in, start_indx, stop_indx):
    """ Devuelve un nuevo dataset mne cortado segun los indices de comienzo y final
    raw_mne_in: dataset mne original
    start_indx, stop_indx: inidces utilizados para cortar el data set segun se necesite
    Return: raw_mne_out es el dataset de salida cortado a partir de raw_mne_in
    """
    data_signal = raw_mne_in.get_data(start=start_indx, stop=stop_indx, return_times=False)
    raw_mne_out = mne.io.RawArray(data_signal, raw_mne_in.info)
    del data_signal

    return raw_mne_out

# Add event by hand
def add_event_manually(raw_mne_in):
    """ Add a custom event to a mne object
    """
    events = mne.find_events(raw_mne_in, shortest_event=1, stim_channel='STIM')
    print(events)


**Tareas posibles**: motor sequence learning (MSL) o visuomotor adaptation (VMA). Por ahora no tengo registros de sueño pero en el futuro podría haber.

**Sesiones**: si bien hay una unica sesion convendría ponerle nombre en caso de que mas adeltante haga mas de una sesión

In [4]:
#Defino los nombres de la tareas y de las sesiones
task = {'msl':"msl",'vma':"vma",'sleep':"sleep"}
session = {'ses1':'day1'}

#Indicar nombre del directorio donde se debe guardar todo el arbol BIDS
bids_root = '/home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS'

Se indica el nombre del archivo nev3 y se crea el reader con la libreria Neo. 

Indicar tambien el numero/codigo del sujeto

In [5]:
subject = '05' # INDICAR CODIGO DE SUJETO 
files_separated = True # ELOS REGISTROS ESTÁN TODOS EN EL MISMO ARCHIVO O EN ARCHIVOS SEPARADOS

En caso de que todo el registro esté en un mismo archivo:

In [6]:
if files_separated == False:
    #INDICAR EL FILE PATH
    nev_filename = '/home/lfa-01/Documentos/CopiaTemp_Testeos_Pacientes_ENYS_BIDS/sub-0XXX/ieeg/'
    neo_reader = load_original_ieeg_nev3(nev_filename)
    print(neo_reader)
else:
    print("Los archivos están separados, se levanta cada uno donde corresponda.")

Los archivos están separados, se levanta cada uno donde corresponda.


Se crea un dataset de mne a partir del reader Neo

In [7]:
if files_separated == False:
    raw_mne = create_mne_dataset_from_neoReader(neo_reader)
    print(raw_mne.info)
else:
    print("Los archivos están separados, se levanta cada uno donde corresponda.")

Los archivos están separados, se levanta cada uno donde corresponda.


Como sanity check se visualizan los eventos registrados para entender si el achivo contiene los entrenamientos msl y vma juntos o quedaron separados. En caso de que esten juntos hay que determinar la muestra que se utlizará para separarlos.

In [8]:
if files_separated == False:
    events = mne.find_events(raw_mne, stim_channel='STIM')
    events
    plt.plot(events[:,0],events[:,2])
    plt.show()
else:
    print("Los archivos están separados, se levanta cada uno donde corresponda.")

Los archivos están separados, se levanta cada uno donde corresponda.


In [13]:
#del raw_mne
#del mne_aux
#del mne_vma
#del mne_msl

### Para gardar dataset como EDF

* Para el registro de VMA

In [12]:
if files_separated == False:
    #INDICAR ESTE VALOR PARA DIVIDIR EL REGISTRO DONDE CORRESPONDA, CHEQUEAR QUE TAREA VA PRIMERO
    max_indx = int(5.297E+6) #se utiliza la muestra/indice que corresponda según como haya quedado estrucutrado el registro
    mne_vma = split_mne_dataset(raw_mne, 0, max_indx)
    print(mne_vma.info)

elif files_separated == True:
    # INDICAR EL NOMBRE EL ARCHIVO CORRESPONDIENTE A LA TAREA VMA
    nev_filename_vma = '/home/lfa-01/Documentos/CopiaTemp_Testeos_Pacientes_ENYS_BIDS/sub-05/ieeg/UcidvjGmlifqkkjymEcf16_03_-20220316-110849-INST0'
    neo_reader_vma = load_original_ieeg_nev3(nev_filename_vma)
    print(neo_reader_vma)

    mne_vma = create_mne_dataset_from_neoReader(neo_reader_vma)
    print(mne_vma.info)

BlackrockRawIO: /home/lfa-01/Documentos/CopiaTemp_Testeos_Pacientes_ENYS_BIDS/sub-05/ieeg/UcidvjGmlifqkkjymEcf16_03_-20220316-110849-INST0
nb_block: 1
nb_segment:  [1]
signal_streams: [nsx3 (chans: 64)]
signal_channels: [TPI1, TPI2, TPI3, INSI1 ... HQD4 , HQD5 , HQD6 , HQD7]
spike_channels: [ch65#0, ch66#0, ch67#0, ch68#0 ... ch109#0 , ch110#0 , ch111#0 , ch112#0]
event_channels: [digital_input_port, serial_input_port, comments]

Creating RawArray with float64 data, n_channels=64, n_times=5899191
    Range : 0 ... 5899190 =      0.000 ...  2949.595 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=5899191
    Range : 0 ... 5899190 =      0.000 ...  2949.595 secs
Ready.
<Info | 8 non-empty values
 bads: []
 ch_names: TPI1, TPI2, TPI3, INSI1, INSI2, INSI3, INSI4, ECG1, ECG2, FOI3, ...
 chs: 62 sEEG, 2 ECG, 1 Stimulus
 custom_ref_applied: False
 dig: 0 items
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 65
 projs: []
 sfreq: 2000.0 Hz
>


In [7]:
del neo_reader_vma

Como sanity check se visualizan los eventos registrados

In [8]:
events_vma = mne.find_events(mne_vma, shortest_event=1, stim_channel='STIM')
print(events_vma)
plt.plot(events_vma[:,0],events_vma[:,2])
plt.show()

1284 events found
Event IDs: [  2   4   8  16  32  64 128]
[[1336226       0      64]
 [1337847       0     128]
 [1339458       0      32]
 ...
 [4985983       0      16]
 [4986383       0      32]
 [5077503       0       2]]


Guardado del registro correspondiente con extensión EDF y en formato BIDS

In [ ]:
task_name = task['vma']
session_name = session['ses1']
run = None
datatype = 'ieeg'
BIDS_path = export_mne_to_edf_BIDSformat(mne_vma, bids_root, subject, task_name, session_name, run, datatype, extension='.edf')

print(BIDS_path.fpath)
print(BIDS_path.root)
print(BIDS_path.extension)
print(BIDS_path.directory)
print(BIDS_path.basename)

In [10]:
del mne_vma

#### CASO ESPECIAL
Para el sujeto 05 hay muchos canales y tengo problemas para guardar su info de una sola vez, voy a separar el registro en canales de Hipocampo + STIM y demas canales + STIM

In [13]:
mne_vma.info['ch_names']
first_letter = 'H'

# Words starting with specific letter
chan_hipp = []
chan_no_hipp = []

for i in mne_vma.info['ch_names']:
    if(i.find(first_letter) == 0 or i.find(first_letter.lower()) == 0):
        chan_hipp.append(i)
    else:
        chan_no_hipp.append(i)

# Se agrega canal 'STIM'
chan_hipp.append('STIM')

print(chan_hipp)
print(chan_no_hipp)
print(len(chan_no_hipp)+len(chan_hipp))

['HKI1', 'HKI2', 'HKI3', 'HKI4', 'HKI5', 'HKI6', 'HQI1', 'HQI2', 'HQI3', 'HQI4', 'HQI5', 'HQI6', 'HKD1', 'HKD2', 'HKD3', 'HKD4', 'HKD5', 'HKD6', 'HKD7', 'HKD8', 'HQD1', 'HQD2', 'HQD3', 'HQD4', 'HQD5', 'HQD6', 'HQD7', 'STIM']
['TPI1', 'TPI2', 'TPI3', 'INSI1', 'INSI2', 'INSI3', 'INSI4', 'ECG1', 'ECG2', 'FOI3', 'FOI4', 'AMI1', 'AMI2', 'AMI3', 'AMI4', 'AMI5', 'AMI6', 'TPD1', 'TPD2', 'TPD3', 'TPD4', 'INSD1', 'INSD2', 'INSD3', 'INSD4', 'INSD5', 'INSD6', 'FOD1', 'FOD2', 'FOD3', 'FOD4', 'AMD1', 'AMD2', 'AMD3', 'AMD4', 'AMD5', 'AMD6', 'STIM']
66


In [14]:
task_name = task['vma']
session_name = session['ses1']
run = None
datatype = 'ieeg' 
description = 'chann_hipp'

mne_vma_select = mne_vma.pick_channels(chan_hipp)
mne_vma.info['ch_names']

: 

: 

In [ ]:
BIDS_path = export_mne_to_edf_BIDSformat(mne_vma_select, bids_root, subject, task_name, session_name, run, datatype, description = description, extension='.edf')

print(BIDS_path.fpath)
print(BIDS_path.root)
print(BIDS_path.extension)
print(BIDS_path.directory)
print(BIDS_path.basename)

In [ ]:
del mne_vma_select

* Para el registro de MSL

In [7]:
if files_separated == False:
    #INDICAR ESTE VALOR PARA DIVIDIR EL REGISTRO DONDE CORRESPONDA, CHEQUEAR QUE TAREA VA PRIMERO
    max_indx = int(5.297E+6) #se utiliza la muestra/indice que corresponda según como haya quedado estrucutrado el registro
    mne_msl = split_mne_dataset(raw_mne, max_indx, None)
    print(mne_msl.info)

elif files_separated == True:
    # INDICAR EL NOMBRE EL ARCHIVO CORRESPONDIENTE A LA TAREA MSL
    nev_filename_msl = '/home/lfa-01/Documentos/CopiaTemp_Testeos_Pacientes_ENYS_BIDS/sub-05/ieeg/UcidvjGmlifqkkjymEcf14_03_-20220315-111802-INST0'
    neo_reader_msl = load_original_ieeg_nev3(nev_filename_msl)
    print(neo_reader_msl)

    mne_msl = create_mne_dataset_from_neoReader(neo_reader_msl)
    print(mne_msl.info)

BlackrockRawIO: /home/lfa-01/Documentos/CopiaTemp_Testeos_Pacientes_ENYS_BIDS/sub-05/ieeg/UcidvjGmlifqkkjymEcf14_03_-20220315-111802-INST0
nb_block: 1
nb_segment:  [1]
signal_streams: [nsx3 (chans: 64)]
signal_channels: [TPI1, TPI2, TPI3, INSI1 ... HQD4 , HQD5 , HQD6 , HQD7]
spike_channels: [ch65#0, ch66#0, ch67#0, ch68#0 ... ch109#0 , ch110#0 , ch111#0 , ch112#0]
event_channels: [digital_input_port, serial_input_port, comments]

Creating RawArray with float64 data, n_channels=64, n_times=6098424
    Range : 0 ... 6098423 =      0.000 ...  3049.211 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=6098424
    Range : 0 ... 6098423 =      0.000 ...  3049.211 secs
Ready.
<Info | 8 non-empty values
 bads: []
 ch_names: TPI1, TPI2, TPI3, INSI1, INSI2, INSI3, INSI4, FOI1, FOI2, FOI3, ...
 chs: 64 sEEG, 1 Stimulus
 custom_ref_applied: False
 dig: 0 items
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 65
 projs: []
 sfreq: 2000.0 Hz
>


In [8]:
del neo_reader_msl 

In [9]:
events_msl = mne.find_events(mne_msl, stim_channel='STIM')
events_msl
plt.plot(events_msl[:,0],events_msl[:,2])
plt.show()

67 events found
Event IDs: [  4   8  16  64 128]


Guardado del registro correspondiente con extensión EDF y en formato BIDS

In [14]:
task_name = task['msl']
session_name = session['ses1']
run = None
datatype = 'ieeg'
BIDS_path = export_mne_to_edf_BIDSformat(mne_msl, bids_root, subject, task_name, session_name, run, datatype, extension='.edf')

print(BIDS_path.fpath)
print(BIDS_path.root)
print(BIDS_path.extension)
print(BIDS_path.directory)
print(BIDS_path.basename)

67 events found
Event IDs: [  4   8  16  64 128]
Writing '/home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS/participants.tsv'...
Writing '/home/lfa-01/Documentos/Datos_Proyecto_LFA-ENYS_BIDS/participants.json'...


: 

: 

In [22]:
del mne_msl